In [5]:
from goprocam import GoProCamera, constants
from threading import Thread
import cv2
import socket
import math
import numpy as np
from time import time
WRITE = False
#setup the tracker, read more https://ehsangazar.com/object-tracking-with-opencv-fd18ccdd7369
tracker = cv2.TrackerKCF_create()
#stream the video from webcam
cap=cv2.VideoCapture(1)
ret,frame = cap.read()
center = []
frame_index = 0    
#select a bounding box to do object tracking
bbox = cv2.selectROI(frame, False,False)
# Initialize tracker with first frame and bounding box
ok = tracker.init(frame, bbox)
while True:
    ret, frame = cap.read()
    cv2.imshow("webcam", frame)
   # hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
   # mask = cv2.inRange(hsv,lower_bound,upper_bound)
   # res = cv2.bitwise_and(frame,frame,mask=mask)
   # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
   # cv2.imshow("res",res)
   # gray = cv2.medianBlur(gray,21)
   # kernel = np.ones((5, 5), np.uint8)
   # mask = cv2.erode(gray, kernel)
# Update tracker
    ok, bbox = tracker.update(frame)
    if ok:
         # Tracking success
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
        center.append([frame_index, int(bbox[0] + bbox[2]/2), int(bbox[1] + bbox[3]/2)])
        #draw center
        cv2.circle(frame,(int(bbox[0] + bbox[2]/2),int(bbox[1] + bbox[3]/2)),2,(0,0,255),3)
    else :
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
    cv2.imshow("tracking",frame)
    np.savetxt("web_cam.csv", center, delimiter=",", fmt='%d', header= "Frame Number, X, Y", comments = '')
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [10]:
import cv2
import socket
import math
import imutils
import numpy as np
from time import time
WRITE = False
center1 = [] #has to differ than the previous cut
frame_index = 0
#define lower and upper boundries for black color
lower_bound = np.array([0,0,0])
upper_bound = np.array([96,96,96])#before it's 96,96,96
#stream the video from webcam
cap=cv2.VideoCapture(0)
while True:
    ret,frame = cap.read()
    cv2.imshow("webcam",frame)
    #blur the picture and transform it to hsv
    blurred = cv2.GaussianBlur(frame,(11,11),0)
    hsv = cv2.cvtColor(blurred,cv2.COLOR_BGR2HSV)
    #construct a mask of black
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    mask = cv2.erode(mask, None, iterations = 2)
    mask = cv2.dilate(mask, None, iterations = 2)
    #find contours in the mask
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnts = imutils.grab_contours(cnts)
    for cn in cnts:
        c = max(cn, key=cv2.contourArea)
        ((x,y),radius) = cv2.minEnclosingCircle(cn)
        if radius > 15:#this may change due to the size of mark
            cv2.circle(frame,(int(x),int(y)),int(radius),(255,255,255),-1)
            center1.append([frame_index, int(x), int(y)])
            #print("frame is",frame_index,"x is", int(x), "y is", int(y))
    frame_index = frame_index + 1
         #cv2.drawContours(frame,[c],-1,(255,255,255),2)
    cv2.imshow("detect",frame)
    #Write modified video to output csv file
    np.savetxt("web_cam1.csv", center1, delimiter=",", fmt='%d', header= "Frame Number, X, Y", comments = '')#the csv file name and array should be matched
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()